In [25]:
%pip install nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from sklearn.metrics.pairwise import cosine_similarity

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
data = 'preprocessed_april25th.csv'

model_data = pd.read_csv(data, usecols=["book_title", "book_desc", "image_url", "book_title_init", "book_authors_init"])
model_data1 = model_data[:3000]
model_data1.head()

,book_desc,book_title,image_url,book_authors_init,book_title_init
0,winning will make you famous losing means cert...,the_hunger_games,https://images.gr-assets.com/books/1447303603l...,Suzanne Collins,The Hunger Games
1,there is a door at the end of a silent corrido...,harry_potter_and_the_order_of_the_phoenix,https://images.gr-assets.com/books/1255614970l...,J.K. Rowling|Mary GrandPré,Harry Potter and the Order of the Phoenix
2,the unforgettable novel of a childhood in a sl...,to_kill_a_mockingbird,https://images.gr-assets.com/books/1361975680l...,Harper Lee,To Kill a Mockingbird
3,about three things i was absolutely positive f...,twilight1,https://images.gr-assets.com/books/1361039443l...,Stephenie Meyer,Twilight
4,trying to make sense of the horrors of world w...,the_book_thief1,https://images.gr-assets.com/books/1522157426l...,Markus Zusak,The Book Thief


In [38]:
model_data1[model_data1["book_title"]=="champion"]

,book_desc,book_title,image_url,book_authors_init,book_title_init
608,he is a legend she is a prodigy who will be ch...,champion,https://images.gr-assets.com/books/1382652310l...,Marie Lu,Champion


In [39]:
tfidf = TfidfVectorizer(analyzer = 'word',
                        min_df=1,
                        max_df = 0.99,
                        stop_words="english",
                        encoding = 'utf-8', 
                        token_pattern=r"(?u)\S\S+")
tfidf_encoding = tfidf.fit_transform(model_data1["book_desc"])
print(tfidf.get_feature_names_out()[1:100])
print(tfidf_encoding.shape)

['001' '007' '01' '0312984820' '0345418921' '0375869026' '038572'
 '0451456734' '05' '057' '0571225381' '0618260269' '0618346252'
 '0679734511' '0743273567' '0743482832' '0785263705' '10' '100' '1048'
 '105' '109' '10th' '11' '112' '1131' '1135' '114' '114d' '118a' '11th'
 '12' '120' '1204' '121' '123' '1265' '12ashton' '13' '1300' '1321' '1327'
 '1330' '1351' '13th' '14' '140' '1400s' '1416524525' '1418' '1439107955'
 '1460' '1478' '1480' '1492' '14th' '15' '150' '150th' '1516' '1520s'
 '1533' '1535' '1547' '1553' '1557' '1564' '1572' '1586' '159' '1597'
 '1599' '15th' '16' '1600' '1603' '1620' '1620s' '1623' '1655' '1660s'
 '1666' '1689' '16th' '17' '1708' '1714' '1726' '1728' '1733' '1740'
 '1743' '1746' '1755' '1767' '1771' '1772' '1774' '1776']
(3000, 26946)


In [40]:
## Find most books with similar words in a given text query

def comp_description(query, results_number=3):
        results=[]
        q_vector = tfidf.transform([query])
        print("Comparable Description: ", query)
        results.append(cosine_similarity(q_vector, tfidf_encoding.toarray()))
        elem_list=[]
        for i in results[:10]:
            for elem in i[0]:
                    #print("Review",f, "Similarity: ", elem)
                    elem_list.append(elem)
                    
        for i in range(results_number):
            print("Most relevant to the query is Book #" ,elem_list.index(max(elem_list)))
            print("Similarity: ", max(elem_list))
            if sum(elem_list) / len(elem_list)==0.0:
                print("No similar descriptions")
            else:
                print(model_data['book_title'].loc[elem_list.index(max(elem_list)):elem_list.index(max(elem_list))], '\n')
                elem_list.pop(elem_list.index(max(elem_list)))

In [41]:
query = "I want to read a Jane Austen book that have gothic elements."
comp_description(query)

Comparable Description:  I want to read a Jane Austen book that have gothic elements.
Most relevant to the query is Book # 2238
Similarity:  0.29311792060052066
2238    austenland
Name: book_title, dtype: object 

Most relevant to the query is Book # 443
Similarity:  0.275368550820846
443    northanger_abbey1
Name: book_title, dtype: object 

Most relevant to the query is Book # 17
Similarity:  0.25635328770926785
17    jane_eyre1
Name: book_title, dtype: object 

